In [5]:
class Node():
    def __init__(self, key = None):
        self.key = key
        self.next = None
        self.previous = None

In [6]:
class DoublyLinkedList():
    def __init__(self):
        self.head = None
        self.tail = None
        
    def add_to_front(self, key, node):  
        # if there is no node to add make a new node using the key
        if node == None:
            node = Node(key)
        # if there are no head and tail in the doubly linked list, 
        # make the new node as head and tail of the doubly linked list
        if self.head == None:
            self.head = node
            self.tail = node
        # if the head exists, make the new node its previous node.
        # make head next of the new node
        # finally make new node as the head node.
        else:
            self.head.previous = node
            node.next = self.head
            self.head = node
        return node
    
    def move_to_front(self, node):
        # if new node is head, do not move.
        if node == self.head:
            return
        # if new node is tail, return the last node
        elif node == self.tail:
            self.remove_last()
        else:
        # if new node is in the middle, return that node
            self.remove_node(node)
        return self.add_to_front(None, node)
    
    def remove_node(self, node):
        # Identify the before and after nodes of the node to remove.
        # Point next of the before node to after node
        # Point previous of the after node to before node
        before_node = node.previous
        after_node = node.next
        before_node.next = after_node
        after_node.previous = before_node
        
    def remove_last(self):
        # Identify the last node. Set tail as previous of the last node.
        # Make next of the tail to point to None.
        last_node = self.tail
        self.tail = last_node.previous
        self.tail.next = None
        return last_node.key
    
    def print_ll(self):
        node = self.head
        llist = []
        while node is not None:
            llist.append(node.key)
            node = node.next
        return llist


In [7]:
class LRU_Cache(object):
    """
    Using doubly linked list to build the cache from hash_map as dictionary
    """

    def __init__(self, capacity):
        self.capacity = capacity
        self.num_of_entries = 0
        self.hash_map = {}
        self.dlinkedlist = DoublyLinkedList()

    def get(self, key):
        # check if the key is hash map
        if key not in self.hash_map:
            return -1
        else:
            # if key is found, get the node from hash_map for that key
            node = self.hash_map[key]['ref']
            # Move the corresponding node to front in the doubly linked list
            self.dlinkedlist.move_to_front(node)
            # return the value of hash_map key
            return self.hash_map[key]['value']
           
    def set(self, key, value):
        # set or update the doubly linked list cache.
        if key == None:
            return
        if self.get(key) == -1:
             # if key is not in hash_map and there is capacity to add 
            if self.num_of_entries < self.capacity:
                self.num_of_entries += 1
                # create a new node and add it to the doubly linked list
                node = self.dlinkedlist.add_to_front(key, None)
                # update the hash_map key with the value and node
                self.hash_map[key] = {'value': value, 'ref': node}
            else:
                 # if capacity full, remove last key
                key_to_remove = self.dlinkedlist.remove_last()
                if key == key_to_remove:
                    # move the last node to front of the linked list
                    self.dlinkedlist.move_to_front(self.cache[key]['ref'])
                    # update hash_map key with the value
                    self.hash_map[key]['value'] = value
                else:
                    # if key is new key, remove the last key
                    del self.hash_map[key_to_remove]
                    # add node to the doubly linked list
                    node = self.dlinkedlist.add_to_front(key, None)
                    # update hash_map key with the value and node
                    self.hash_map[key] = {'value': value, 'ref': node}
        else:
             # if key found in the dictionary, overwrite the value
            self.hash_map[key]['value'] = value
   

In [24]:
def test_cache_create(capacity, input_list, expected_output, test_details):
    our_cache = LRU_Cache(capacity)
    print("Initialize cache with capacity: {}".format(capacity))
    for key, value in input_list:
        print("Set {}:{}".format(key, value))
        our_cache.set(key, value)
    linked_list = our_cache.dlinkedlist.print_ll()
    if linked_list == expected_output:
        print(test_details+': Pass {}'.format(linked_list))
    else:
        print(test_details+': Fail {}'.format(linked_list))

def test_cache_get_by_element(capacity, input_list, get_element, expected_output, test_details):
    our_cache = LRU_Cache(capacity)
    print("Initialize cache with capacity: {}".format(capacity))
    for key, value in input_list:
        print("Set {}:{}".format(key, value))
        our_cache.set(key, value)
    if our_cache.get(get_element) == expected_output:
        print(test_details+': Pass, Return {} for get({})'.format(expected_output,get_element))
    else:
        print(test_details+': Fail, Actual: {} Vs Expected: {}'.format(our_cache.get(get_element),expected_output))
        



In [14]:
test_details = "create cache from the input_list and compare it with the expected_output"
input_list = [(1, 1),(2, 2),(3, 3),(4, 4)]
expected_output = [4,3,2,1]
capacity = 5
test_cache_set(capacity, input_list, expected_output, test_details)

Initialize cache with capacity: 5
Set 1:1
Set 2:2
Set 3:3
Set 4:4
create cache from the input_list and compare it with the expected_output: Pass [4, 3, 2, 1]


In [15]:
test_details = "Test the None input for key"
input_list = [(1, 1),(None, 2),(3, 3),(4, 4)]
expected_output = [4,3,1]
capacity = 5
test_cache_set(capacity, input_list, expected_output, test_details)


Initialize cache with capacity: 5
Set 1:1
Set None:2
Set 3:3
Set 4:4
Handles None input: Pass [4, 3, 1]


In [16]:
test_details = "Test if LRU entries are purged"
input_list = [(1, 1),(2, 2),(3, 3),(4, 4),(5,5),(6,6),(7,7)]
expected_output = [7,6,5,4,3]
capacity = 5
test_cache_set(capacity, input_list, expected_output, test_details)        

Initialize cache with capacity: 5
Set 1:1
Set 2:2
Set 3:3
Set 4:4
Set 5:5
Set 6:6
Set 7:7
Test if LRU entries are purged: Pass [7, 6, 5, 4, 3]


In [17]:
test_details= 'Test cache miss'
input_list = [(1, 1),(2, 2),(3, 3),(4, 4)]
get_element = 15
expected_output = -1
capacity = 5
test_cache_get_by_element(capacity, input_list, get_element, expected_output, test_details)

Initialize cache with capacity: 5
Set 1:1
Set 2:2
Set 3:3
Set 4:4
Test cache miss: Pass, Return -1 for get(15)


In [18]:
test_details= 'Test cache hit'
input_list = [(1, 1),(2, 2),(3, 3),(4, 4)]
get_element = 4
expected_output = 4
capacity = 5
test_cache_get_by_element(capacity, input_list, get_element, expected_output, test_details)

Initialize cache with capacity: 5
Set 1:1
Set 2:2
Set 3:3
Set 4:4
Test cache hit: Pass, Return 4 for get(4)
